# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
#Load CSV file as DataFrame
city_data = pd.read_csv("../output_data/cities.csv")
city_data.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,qaanaaq,77.4840,-69.3632,43.38,66,81,3.89,GL,1659324369
1,1,genhe,50.7833,121.5167,80.76,39,33,6.26,CN,1659324370
2,2,brewster,41.3973,-73.6171,75.61,83,0,0.00,US,1659324371
3,3,yar-sale,66.8333,70.8333,56.66,61,5,8.59,RU,1659324371
4,4,dikson,73.5069,80.5464,41.29,94,100,9.46,RU,1659324372
5,5,severo-kurilsk,50.6789,156.1250,55.83,76,48,9.86,RU,1659324238
6,6,yashkul,46.1711,45.3435,80.94,50,0,2.64,RU,1659324372
7,7,batticaloa,7.7102,81.6924,80.87,78,99,3.33,LK,1659324373
8,8,villamor,6.7047,124.5186,82.72,75,100,2.77,PH,1659324373
9,9,bambanglipuro,-7.9500,110.2833,82.42,65,50,4.61,ID,1659324374


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
#Set Humidity as Weight
humidity = city_data["Humidity"].astype(float)
maxhumidity = humidity.max()

#Set Lat and Lng as Locations
locations = city_data[["Lat", "Lng"]]

In [18]:
#Plot Map
fig = gmaps.figure()

#Add Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
plt.savefig("../Images/HeatMap.png")

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#Narrow Cities to Reflect Conditions
narrowed_cities = city_data.loc[(city_data["Max Temp"] > 70) & (city_data["Max Temp"] < 80) & (city_data["Cloudiness"] == 0), :]

#Drop Null Values
narrowed_cities = narrowed_cities.dropna(how='any')
narrowed_cities.reset_index(inplace=True)

#Delete Unneccessary Columns
del narrowed_cities['index']
narrowed_cities.head()

#Show DataFrame
narrowed_cities.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,2,brewster,41.3973,-73.6171,75.61,83,0,0.00,US,1659324371
1,40,hamilton,39.1834,-84.5333,71.24,91,0,5.75,US,1659324219
2,147,vila franca do campo,37.7167,-25.4333,70.21,86,0,2.89,PT,1659324423
3,157,bethel,41.3712,-73.4140,75.49,81,0,0.00,US,1659324146
4,174,estepa,37.2926,-4.8790,70.29,78,0,6.33,ES,1659324434
5,189,porangatu,-13.4408,-49.1486,72.10,27,0,2.86,BR,1659324441
6,244,yuzhnyy,53.2542,83.6936,71.98,56,0,15.66,RU,1659324462
7,255,mao,39.8885,4.2658,75.76,78,0,8.05,ES,1659324466
8,327,merritt,50.1123,-120.7942,79.20,33,0,5.79,CA,1659324496
9,344,zuwarah,32.9312,12.0820,79.68,73,0,6.44,LY,1659324503


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
#Create Empty List for Hotels
hotellist = []

#Begin For Loop to Find Hotels
for i in range(len(narrowed_cities)):
    lat = narrowed_cities.loc[i]['Lat']
    lng = narrowed_cities.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_cities["Hotel Name"] = hotellist
narrowed_cities = narrowed_cities.dropna(how='any')

    

In [22]:
narrowed_cities.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
0,2,brewster,41.3973,-73.6171,75.61,83,0,0.00,US,1659324371,Carmel Hamlet
1,40,hamilton,39.1834,-84.5333,71.24,91,0,5.75,US,1659324219,Cincinnati
2,147,vila franca do campo,37.7167,-25.4333,70.21,86,0,2.89,PT,1659324423,Vila Franca do Campo
3,157,bethel,41.3712,-73.4140,75.49,81,0,0.00,US,1659324146,Danbury
4,174,estepa,37.2926,-4.8790,70.29,78,0,6.33,ES,1659324434,Estepa
5,189,porangatu,-13.4408,-49.1486,72.10,27,0,2.86,BR,1659324441,Porangatu
6,244,yuzhnyy,53.2542,83.6936,71.98,56,0,15.66,RU,1659324462,Barnaul
7,255,mao,39.8885,4.2658,75.76,78,0,8.05,ES,1659324466,Mahón
8,327,merritt,50.1123,-120.7942,79.20,33,0,5.79,CA,1659324496,Merritt
9,344,zuwarah,32.9312,12.0820,79.68,73,0,6.44,LY,1659324503,Zuwara


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))